# Submission 2: Advanced Feature Engineering Model

In [ ]:
import pandas as pd
import numpy as np

print('--- Running Submission 2: Advanced Features ---')

# Simulate creating a different submission file
submission_df = pd.DataFrame({
    'battle_id': range(10),
    'prediction': np.random.randint(0, 2, 10) # Different dummy predictions
})

submission_df.to_csv('submission_2.csv', index=False)

print('✅ Submission 2 complete. `submission_2.csv` created.')